In [68]:
import pandas as pd
from datetime import datetime

In [69]:
"""
Mes -> 	Fecha del mes en formato YYYY-MM-DD. Representa el mes al que corresponden los datos climáticos.
temperature_2m_max	-> Temperatura máxima promedio del mes medida a 2 metros de altura, en grados Celsius (°C).
temperature_2m_min -> Temperatura mínima promedio del mes, también a 2 metros de altura, en grados Celsius (°C).
precipitation_sum -> Precipitación total del mes en milímetros (mm). Suma de toda el agua caída durante ese mes.
winddirection_10m_dominant -> Dirección dominante del viento a 10 metros de altura, en grados. Se mide en sentido azimutal desde el norte. (Ej.: 0° = norte, 90° = este, etc.).
SEREMI -> Nombre de la región sanitaria (SEREMI) a la que corresponde esta fila.
"""

'\nMes -> \tFecha del mes en formato YYYY-MM-DD. Representa el mes al que corresponden los datos climáticos.\ntemperature_2m_max\t-> Temperatura máxima promedio del mes medida a 2 metros de altura, en grados Celsius (°C).\ntemperature_2m_min -> Temperatura mínima promedio del mes, también a 2 metros de altura, en grados Celsius (°C).\nprecipitation_sum -> Precipitación total del mes en milímetros (mm). Suma de toda el agua caída durante ese mes.\nwinddirection_10m_dominant -> Dirección dominante del viento a 10 metros de altura, en grados. Se mide en sentido azimutal desde el norte. (Ej.: 0° = norte, 90° = este, etc.).\nSEREMI -> Nombre de la región sanitaria (SEREMI) a la que corresponde esta fila.\n'

### Creación de DataSet's de clima mensual por año y SEREMI:

In [70]:
# 1. Cargar todos los archivos seremi
seremi_data = {
    "2014": pd.read_csv("seremi_2014.csv"),
    "2015": pd.read_csv("seremi_2015.csv"),
    "2016": pd.read_csv("seremi_2016.csv"),
    "2017": pd.read_csv("seremi_2017.csv"),
    "2018": pd.read_csv("seremi_2018.csv"),
    "2019": pd.read_csv("seremi_2019.csv"),
    "2020": pd.read_csv("seremi_2020.csv"),
    "2021": pd.read_csv("seremi_2021.csv"),
    "2022": pd.read_csv("seremi_2022.csv"),
    "2023": pd.read_csv("seremi_2023.csv"),
    "2024": pd.read_csv("seremi_2024.csv"),
    "2025": pd.read_csv("seremi_2025.csv"),
}

# 2. Función para procesar cada dataset climático
def procesar_datasets_climaticos(seremi_data_dict):
    def grados_a_cardinal(grados):
        if pd.isna(grados):
            return None
        direcciones = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
        index = int((grados + 22.5) % 360 // 45)
        return direcciones[index]

    for año, df in seremi_data_dict.items():
        # Eliminar columna innecesaria
        if "Unnamed: 0" in df.columns:
            df = df.drop(columns=["Unnamed: 0"])

        # Crear columna cardinal
        if "winddirection_10m_dominant" in df.columns:
            df["wind_direction_cardinal"] = df["winddirection_10m_dominant"].apply(grados_a_cardinal)

        # Renombrar columnas
        df = df.rename(columns={
            "temperature_2m_max": "Temperatura Máxima",
            "temperature_2m_min": "Temperatura Mínima",
            "precipitation_sum": "Precipitaciones (suma)",
            "winddirection_10m_dominant": "Dirección Viento"
        })

        # Limpiar columna de grados si existe y renombrar cardinal
        if "Dirección Viento" in df.columns:
            df = df.drop(columns=["Dirección Viento"])
        if "wind_direction_cardinal" in df.columns:
            df = df.rename(columns={"wind_direction_cardinal": "Dirección del Viento"})

        # Guardar el dataframe procesado
        seremi_data_dict[año] = df

    return seremi_data_dict

# 3. Aplicar la función a todos los datasets climáticos
seremi_data = procesar_datasets_climaticos(seremi_data)

# 4. Verificar que todo esté correcto (opcional)
seremi_data["2015"].head(12)

,Mes,Temperatura Máxima,Temperatura Mínima,Precipitaciones (suma),SEREMI,Dirección del Viento
0,2015-01-01,12.277419,6.867742,33.7,Magallanes,W
1,2015-02-01,12.142857,7.100000,36.9,Magallanes,W
2,2015-03-01,11.561290,6.393548,92.1,Magallanes,W
3,2015-04-01,9.093333,4.446667,57.9,Magallanes,W
4,2015-05-01,5.935484,2.541935,100.0,Magallanes,S
5,2015-06-01,3.943333,1.150000,92.8,Magallanes,SW
6,2015-07-01,3.032258,-0.138710,98.4,Magallanes,SW
7,2015-08-01,4.180645,0.509677,59.8,Magallanes,SW
8,2015-09-01,5.266667,1.120000,46.0,Magallanes,SW
9,2015-10-01,8.925806,2.803226,24.2,Magallanes,SW


### Creación de DataSet's sobre ocupación de camas por mes y por año según hospital y área:

In [71]:
# Archivos filtrados hospitalarios
archivos_filtrados = {
    "2014": "2014_filtrado.csv",
    "2015": "2015_filtrado.csv",
    "2016": "2016_filtrado.csv",
    "2017": "2017_filtrado.csv",
    "2018": "2018_filtrado.csv",
    "2019": "2019_filtrado.csv",
    "2020": "2020_filtrado.csv",
    "2021": "2021_filtrado.csv",
    "2022": "2022_filtrado.csv",
    "2023": "2023_filtrado.csv",
    "2024": "2024_filtrado.csv",
    "2025": "2025_filtrado.csv"
}

# Diccionario para guardar los dataframes ya limpios
filtrado_limpio = {}

# Abreviaturas de los meses
meses_abreviados = ['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun',
                    'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic']

# Columnas que se eliminarán si existen
columnas_a_eliminar = ["Cód. SS/SEREMI", "Cód. Estab.", "Cód. Nivel Cuidado", "Acum", "Año"]

# Procesamiento de los archivos
for año, archivo in archivos_filtrados.items():
    # Cargar CSV
    df = pd.read_csv(archivo)

    # Eliminar filas irrelevantes
    df = df[~df["Nombre Establecimiento"].isin(["Datos País", "Datos Servicio Salud"])].copy()

    # Eliminar columnas innecesarias
    df = df.drop(columns=columnas_a_eliminar, errors='ignore')

    # Renombrar columnas clave
    df = df.rename(columns={
        "Nombre SS/SEREMI": "SEREMI",
        "Nombre Nivel Cuidado": "Area"
    })

    # Renombrar columnas de meses abreviados por fecha
    nuevo_nombre_columnas = {}
    año_int = int(año)

    for i, mes_abrev in enumerate(meses_abreviados, start=1):
        if mes_abrev in df.columns:
            fecha = datetime(año_int, i, 1).strftime('%Y-%m-%d')
            nuevo_nombre_columnas[mes_abrev] = fecha

    df = df.rename(columns=nuevo_nombre_columnas)

    # Guardar DataFrame limpio
    filtrado_limpio[año] = df

# Ejemplo: verificar que todo cargó bien
filtrado_limpio["2020"].head()


,SEREMI,Nombre Establecimiento,Area,Glosa,2020-01-01,2020-02-01,2020-03-01,2020-04-01,2020-05-01,2020-06-01,2020-07-01,2020-08-01,2020-09-01,2020-10-01,2020-11-01,2020-12-01
24,Antofagasta,Hospital de Mejillones,Datos Establecimiento,Dias Cama Disponibles,279.0,261.0,279.0,270.0,279.0,270.0,279.0,279.0,270.0,270.00,270.0,279.0
25,Antofagasta,Hospital de Mejillones,Datos Establecimiento,Dias Cama Ocupados,79.0,65.0,43.0,256.0,8.0,35.0,45.0,72.0,130.0,74.00,36.0,48.0
26,Antofagasta,Hospital de Mejillones,Datos Establecimiento,Dias de Estada,79.0,65.0,43.0,256.0,8.0,35.0,45.0,72.0,130.0,74.00,36.0,48.0
27,Antofagasta,Hospital de Mejillones,Datos Establecimiento,Promedio Cama Disponibles,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,8.71,9.0,9.0
28,Antofagasta,Hospital de Mejillones,Datos Establecimiento,Numero de Egresos,13.0,21.0,13.0,14.0,4.0,12.0,6.0,10.0,11.0,13.00,11.0,13.0


### Unión de datos:

In [72]:
#
def extraer_info_hospital_area_todo_el_año(hospital_nombre, area_nombre):
    registros = []

    for año, df in filtrado_limpio.items():
        df_seremi = seremi_data[año].copy()

        # Aseguramos que 'Mes' esté en formato datetime
        df_seremi['Mes'] = pd.to_datetime(df_seremi['Mes'], errors='coerce')
        df_seremi.dropna(subset=['Mes'], inplace=True)
        df_seremi['Mes'] = df_seremi['Mes'].dt.to_period("M").dt.to_timestamp()

        columnas_mes = [col for col in df.columns if col.startswith("20")]
        for col in columnas_mes:
            # Filtro hospitalario por hospital, área y mes
            df_tmp = df[
                (df["Nombre Establecimiento"] == hospital_nombre) &
                (df["Area"] == area_nombre)
            ][["SEREMI", "Nombre Establecimiento", "Area", "Glosa", col]].copy()

            if df_tmp.empty:
                continue

            df_tmp = df_tmp.rename(columns={col: "Valor"})
            df_tmp["Fecha"] = pd.to_datetime(col)

            # Filtro climático por seremi y mes
            clima_mes = df_seremi[df_seremi["Mes"] == df_tmp["Fecha"].iloc[0]].copy()
            df_merge = df_tmp.merge(clima_mes, on=["SEREMI"], how="left")

            registros.append(df_merge)

    if registros:
        return pd.concat(registros, ignore_index=True).sort_values("Fecha")
    else:
        print("⚠️ No se encontraron datos para ese hospital y área.")
        return pd.DataFrame()

# Ejemplo de prueba
ejemplo_df = extraer_info_hospital_area_todo_el_año(
    "Hospital Barros Luco Trudeau (Santiago, San Miguel)",
    "401 - Área Médica Adulto Cuidados Básicos"
)

dataframe=ejemplo_df

dataframe.head()


           


,SEREMI,Nombre Establecimiento,Area,Glosa,Valor,Fecha,Mes,Temperatura Máxima,Temperatura Mínima,Precipitaciones (suma),Dirección del Viento
0,Metropolitano Sur,"Hospital Barros Luco Trudeau (Santiago, San Mi...",401 - Área Médica Adulto Cuidados Básicos,Dias Cama Disponibles,3469.0,2016-01-01,2016-01-01,28.664516,16.0,11.2,SW
1,Metropolitano Sur,"Hospital Barros Luco Trudeau (Santiago, San Mi...",401 - Área Médica Adulto Cuidados Básicos,Dias Cama Ocupados,3135.0,2016-01-01,2016-01-01,28.664516,16.0,11.2,SW
2,Metropolitano Sur,"Hospital Barros Luco Trudeau (Santiago, San Mi...",401 - Área Médica Adulto Cuidados Básicos,Dias de Estada,2879.0,2016-01-01,2016-01-01,28.664516,16.0,11.2,SW
3,Metropolitano Sur,"Hospital Barros Luco Trudeau (Santiago, San Mi...",401 - Área Médica Adulto Cuidados Básicos,Promedio Cama Disponibles,111.9,2016-01-01,2016-01-01,28.664516,16.0,11.2,SW
4,Metropolitano Sur,"Hospital Barros Luco Trudeau (Santiago, San Mi...",401 - Área Médica Adulto Cuidados Básicos,Numero de Egresos,332.0,2016-01-01,2016-01-01,28.664516,16.0,11.2,SW


In [73]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [74]:
def preparar_dataset_climatico(df):
    df_pivot = df.pivot(index='Fecha', columns='Glosa', values='Valor').reset_index()

    columnas_necesarias = ['Dias Cama Disponibles', 'Dias Cama Ocupados',
                           'Promedio Cama Disponibles', 'Numero de Egresos']
    if not all(col in df_pivot.columns for col in columnas_necesarias):
        print("❌ Datos insuficientes para generar el modelo.")
        return None

    clima = df.drop_duplicates(subset='Fecha')[[
        'Fecha', 'Temperatura Máxima', 'Temperatura Mínima',
        'Precipitaciones (suma)', 'Dirección del Viento'
    ]]

    df_pivot = pd.merge(df_pivot, clima, on='Fecha', how='left')
    df_pivot = pd.get_dummies(df_pivot, columns=['Dirección del Viento'], prefix='Viento')

    df_pivot = df_pivot.sort_values('Fecha')
    df_pivot['Mes'] = df_pivot['Fecha'].dt.month
    df_pivot['Trimestre'] = df_pivot['Fecha'].dt.quarter
    df_pivot['lag_1'] = df_pivot['Dias Cama Disponibles'].shift(1)
    df_pivot['lag_2'] = df_pivot['Dias Cama Disponibles'].shift(2)
    df_pivot['porcentaje_ocupacion'] = df_pivot['Dias Cama Ocupados'] / df_pivot['Dias Cama Disponibles']
    df_pivot['variacion_disponibles'] = df_pivot['Dias Cama Disponibles'].diff()
    df_pivot['ocupados_media_movil'] = df_pivot['Dias Cama Ocupados'].rolling(window=3).mean()
    df_pivot['promedio_media_movil'] = df_pivot['Promedio Cama Disponibles'].rolling(window=3).mean()
    df_pivot['egresos_media_movil'] = df_pivot['Numero de Egresos'].rolling(window=3).mean()

    return df_pivot.dropna().copy()


In [51]:
# Pipeline final robusto con clima + hospital + evaluación completa y errores porcentuales

def pipeline_final_mae_reducido(hospital, area):
    print(f"🔍 Extrayendo datos para: {hospital} | {area}")
    df_base = extraer_info_hospital_area_todo_el_año(hospital, area)

    if df_base.empty:
        print("❌ No se encontraron datos para el hospital o área especificada.")
        return None

    print("🧪 Preparando dataset con variables hospitalarias y climáticas...")
    df_modelo = preparar_dataset_climatico(df_base)

    if df_modelo is None or df_modelo.empty:
        print("❌ No se pudo preparar el dataset.")
        return None

    # Definir features dinámicamente
    features = [
        'Dias Cama Ocupados',
        'Promedio Cama Disponibles',
        'Numero de Egresos',
        'Mes', 'Trimestre',
        'lag_1', 'lag_2',
        'porcentaje_ocupacion',
        'variacion_disponibles',
        'ocupados_media_movil',
        'promedio_media_movil',
        'egresos_media_movil',
        'Temperatura Máxima',
        'Temperatura Mínima',
        'Precipitaciones (suma)'
    ] + [col for col in df_modelo.columns if col.startswith('Viento_')]

    # Separar entrenamiento y prueba (último 20% como evaluación)
    X = df_modelo[features]
    y = df_modelo['Dias Cama Disponibles']
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

    print("🔧 Ajustando modelo con GridSearchCV...")
    param_grid = {
        'n_estimators': [50, 100],
        'max_depth': [2, 3],
        'learning_rate': [0.05, 0.1]
    }

    model = XGBRegressor(n_jobs=1, random_state=42)
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_absolute_error',
        cv=TimeSeriesSplit(n_splits=3),
        verbose=1
    )
    grid_search.fit(X_train, y_train)
    mejor_modelo = grid_search.best_estimator_
    print("📌 Mejores parámetros encontrados:", grid_search.best_params_)

    print("📊 Evaluando sobre el 20% final de los datos...")

    y_pred = mejor_modelo.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    error_pct = np.mean(np.abs((y_test.values - y_pred) / y_test.values)) * 100

    resultados = pd.DataFrame({
        'Fecha': df_modelo['Fecha'].iloc[-len(y_test):].values,
        'Real': y_test.values,
        'Predicho': y_pred,
        'Error Absoluto': np.abs(y_test.values - y_pred),
        'Error Porcentual (%)': np.abs((y_test.values - y_pred) / y_test.values) * 100
    })

    print(f"✅ MAE (camas): {mae:.2f}")
    print(f"✅ RMSE (camas): {rmse:.2f}")
    print(f"📉 Error promedio porcentual: {error_pct:.2f}%")

    return resultados




In [52]:
resultados_final = pipeline_final_mae_reducido(
    "Hospital Barros Luco Trudeau (Santiago, San Miguel)",
    "401 - Área Médica Adulto Cuidados Básicos"
)

display(resultados_final)


🔍 Extrayendo datos para: Hospital Barros Luco Trudeau (Santiago, San Miguel) | 401 - Área Médica Adulto Cuidados Básicos
🧪 Preparando dataset con variables hospitalarias y climáticas...
🔧 Ajustando modelo con GridSearchCV...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
📌 Mejores parámetros encontrados: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 66.96
✅ RMSE (camas): 87.72
📉 Error promedio porcentual: 3.04%


,Fecha,Real,Predicho,Error Absoluto,Error Porcentual (%)
0,2023-05-01,2329.0,2254.683350,74.316650,3.190925
1,2023-06-01,2250.0,2236.470215,13.529785,0.601324
2,2023-07-01,2325.0,2259.773926,65.226074,2.805423
3,2023-08-01,2325.0,2269.901367,55.098633,2.369834
4,2023-09-01,2252.0,2256.211670,4.211670,0.187019
5,2023-10-01,2334.0,2273.539795,60.460205,2.590412
6,2023-11-01,2252.0,2250.232910,1.767090,0.078468
7,2023-12-01,2144.0,2104.215576,39.784424,1.855617
8,2024-01-01,2146.0,2111.233154,34.766846,1.620077
9,2024-02-01,2035.0,2155.288574,120.288574,5.910986


In [ ]:
##############################
##############################
##DESDE AQUI, NUEVAS PRUEBAS##
##############################
##############################

In [53]:
# Ampliamos la función `preparar_dataset_climatico` con nuevas features derivados:
# - tasas
# - cruces entre clima y variables hospitalarias
# - más variaciones

def preparar_dataset_climatico(df):
    df_pivot = df.pivot(index='Fecha', columns='Glosa', values='Valor').reset_index()

    columnas_necesarias = ['Dias Cama Disponibles', 'Dias Cama Ocupados',
                           'Promedio Cama Disponibles', 'Numero de Egresos']
    if not all(col in df_pivot.columns for col in columnas_necesarias):
        print("❌ Datos insuficientes para generar el modelo.")
        return None

    # Añadir clima
    clima = df.drop_duplicates(subset='Fecha')[[
        'Fecha', 'Temperatura Máxima', 'Temperatura Mínima',
        'Precipitaciones (suma)', 'Dirección del Viento'
    ]]

    df_pivot = pd.merge(df_pivot, clima, on='Fecha', how='left')
    df_pivot = pd.get_dummies(df_pivot, columns=['Dirección del Viento'], prefix='Viento')

    # Ordenar temporalmente
    df_pivot = df_pivot.sort_values('Fecha')

    # Variables temporales básicas
    df_pivot['Mes'] = df_pivot['Fecha'].dt.month
    df_pivot['Trimestre'] = df_pivot['Fecha'].dt.quarter
    df_pivot['lag_1'] = df_pivot['Dias Cama Disponibles'].shift(1)
    df_pivot['lag_2'] = df_pivot['Dias Cama Disponibles'].shift(2)
    df_pivot['porcentaje_ocupacion'] = df_pivot['Dias Cama Ocupados'] / df_pivot['Dias Cama Disponibles']
    df_pivot['variacion_disponibles'] = df_pivot['Dias Cama Disponibles'].diff()
    df_pivot['ocupados_media_movil'] = df_pivot['Dias Cama Ocupados'].rolling(window=3).mean()
    df_pivot['promedio_media_movil'] = df_pivot['Promedio Cama Disponibles'].rolling(window=3).mean()
    df_pivot['egresos_media_movil'] = df_pivot['Numero de Egresos'].rolling(window=3).mean()

    # 🔧 Nuevas features derivadas:
    df_pivot['tasa_egresos'] = df_pivot['Numero de Egresos'] / df_pivot['Dias Cama Ocupados']
    df_pivot['tasa_ocupacion_promedio'] = df_pivot['Dias Cama Ocupados'] / df_pivot['Promedio Cama Disponibles']
    df_pivot['variacion_ocupados'] = df_pivot['Dias Cama Ocupados'].diff()
    df_pivot['variacion_egresos'] = df_pivot['Numero de Egresos'].diff()

    # 🔁 Cruces con clima
    df_pivot['ocupados_x_temp'] = df_pivot['Dias Cama Ocupados'] * df_pivot['Temperatura Máxima']
    df_pivot['egresos_x_precipitacion'] = df_pivot['Numero de Egresos'] * df_pivot['Precipitaciones (suma)']
    df_pivot['ocupacion_pct_x_tempmin'] = df_pivot['porcentaje_ocupacion'] * df_pivot['Temperatura Mínima']

    return df_pivot.dropna().copy()


In [54]:
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
import numpy as np

def pipeline_final_mae_reducido(hospital, area):
    print(f"🔍 Extrayendo datos para: {hospital} | {area}")
    df_base = extraer_info_hospital_area_todo_el_año(hospital, area)

    if df_base.empty:
        print("❌ No se encontraron datos para el hospital o área especificada.")
        return None

    print("🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...")
    df_modelo = preparar_dataset_climatico(df_base)

    if df_modelo is None or df_modelo.empty:
        print("❌ No se pudo preparar el dataset.")
        return None

    # Features dinámicos
    features = [
        'Dias Cama Ocupados',
        'Promedio Cama Disponibles',
        'Numero de Egresos',
        'Mes', 'Trimestre',
        'lag_1', 'lag_2',
        'porcentaje_ocupacion',
        'variacion_disponibles',
        'ocupados_media_movil',
        'promedio_media_movil',
        'egresos_media_movil',
        'tasa_egresos',
        'tasa_ocupacion_promedio',
        'variacion_ocupados',
        'variacion_egresos',
        'ocupados_x_temp',
        'egresos_x_precipitacion',
        'ocupacion_pct_x_tempmin',
        'Temperatura Máxima',
        'Temperatura Mínima',
        'Precipitaciones (suma)'
    ] + [col for col in df_modelo.columns if col.startswith('Viento_')]

    # Separar entrenamiento y test (último 20% como test)
    X = df_modelo[features]
    y = df_modelo['Dias Cama Disponibles']
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

    print("🔧 Ajustando modelo con GridSearchCV...")
    param_grid = {
        'n_estimators': [50, 100],
        'max_depth': [2, 3],
        'learning_rate': [0.05, 0.1]
    }

    model = XGBRegressor(n_jobs=1, random_state=42)
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_absolute_error',
        cv=TimeSeriesSplit(n_splits=3),
        verbose=1
    )
    grid_search.fit(X_train, y_train)
    mejor_modelo = grid_search.best_estimator_

    print("📌 Mejores parámetros encontrados:", grid_search.best_params_)

    print("📊 Evaluando sobre el 20% final de los datos...")

    y_pred = mejor_modelo.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    error_pct = np.mean(np.abs((y_test.values - y_pred) / y_test.values)) * 100

    resultados = pd.DataFrame({
        'Fecha': df_modelo['Fecha'].iloc[-len(y_test):].values,
        'Real': y_test.values,
        'Predicho': y_pred,
        'Error Absoluto': np.abs(y_test.values - y_pred),
        'Error Porcentual (%)': np.abs((y_test.values - y_pred) / y_test.values) * 100
    })

    print(f"✅ MAE (camas): {mae:.2f}")
    print(f"✅ RMSE (camas): {rmse:.2f}")
    print(f"📉 Error promedio porcentual: {error_pct:.2f}%")

    return resultados


In [55]:
hospital = "Hospital Barros Luco Trudeau (Santiago, San Miguel)"
area = "401 - Área Médica Adulto Cuidados Básicos"

resultados_mejorado = pipeline_final_mae_reducido(hospital, area)

if resultados_mejorado is not None:
    display(resultados_mejorado)


🔍 Extrayendo datos para: Hospital Barros Luco Trudeau (Santiago, San Miguel) | 401 - Área Médica Adulto Cuidados Básicos
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
📌 Mejores parámetros encontrados: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 67.76
✅ RMSE (camas): 88.44
📉 Error promedio porcentual: 3.06%


,Fecha,Real,Predicho,Error Absoluto,Error Porcentual (%)
0,2023-05-01,2329.0,2265.841309,63.158691,2.711837
1,2023-06-01,2250.0,2222.516602,27.483398,1.221484
2,2023-07-01,2325.0,2253.972656,71.027344,3.054940
3,2023-08-01,2325.0,2261.367676,63.632324,2.736874
4,2023-09-01,2252.0,2246.199951,5.800049,0.257551
5,2023-10-01,2334.0,2260.542480,73.457520,3.147280
6,2023-11-01,2252.0,2241.694092,10.305908,0.457634
7,2023-12-01,2144.0,2106.540771,37.459229,1.747166
8,2024-01-01,2146.0,2122.444336,23.555664,1.097654
9,2024-02-01,2035.0,2146.662354,111.662354,5.487094


In [56]:
# Lista de hospitales y área para evaluar con el mismo pipeline robusto
hospitales_a_evaluar = [
    ("Hospital Barros Luco Trudeau (Santiago, San Miguel)", "401 - Área Médica Adulto Cuidados Básicos"),
    ("Complejo Hospitalario San José (Santiago, Independencia)", "401 - Área Médica Adulto Cuidados Básicos"),
    ("Hospital Carlos Van Buren (Valparaíso)", "401 - Área Médica Adulto Cuidados Básicos"),
    ("Hospital Clínico Herminda Martín (Chillán)", "401 - Área Médica Adulto Cuidados Básicos"),
    ("Hospital El Pino (Santiago, San Bernardo)", "401 - Área Médica Adulto Cuidados Básicos"),
]

# Diccionario para almacenar resultados
resultados_todos = {}

# Evaluar cada hospital
for hospital, area in hospitales_a_evaluar:
    print("\n==========================")
    print(f"🏥 Evaluando: {hospital}")
    print("==========================")
    resultados = pipeline_final_mae_reducido(hospital, area)
    if resultados is not None:
        resultados_todos[hospital] = resultados

# Mostrar un ejemplo del primer hospital evaluado
primer_hospital = list(resultados_todos.keys())[0]
resultados_todos[primer_hospital].head()



🏥 Evaluando: Hospital Barros Luco Trudeau (Santiago, San Miguel)
🔍 Extrayendo datos para: Hospital Barros Luco Trudeau (Santiago, San Miguel) | 401 - Área Médica Adulto Cuidados Básicos
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
📌 Mejores parámetros encontrados: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 67.76
✅ RMSE (camas): 88.44
📉 Error promedio porcentual: 3.06%

🏥 Evaluando: Complejo Hospitalario San José (Santiago, Independencia)
🔍 Extrayendo datos para: Complejo Hospitalario San José (Santiago, Independencia) | 401 - Área Médica Adulto Cuidados Básicos
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
📌 Mejores parámetros encontrados: {'learning_ra

,Fecha,Real,Predicho,Error Absoluto,Error Porcentual (%)
0,2023-05-01,2329.0,2265.841309,63.158691,2.711837
1,2023-06-01,2250.0,2222.516602,27.483398,1.221484
2,2023-07-01,2325.0,2253.972656,71.027344,3.054940
3,2023-08-01,2325.0,2261.367676,63.632324,2.736874
4,2023-09-01,2252.0,2246.199951,5.800049,0.257551


In [57]:
# Función para extraer todos los pares únicos (hospital, área) desde los datos combinados
def extraer_hospitales_y_areas_validos(combinados):
    pares_hospital_area = set()

    for df in combinados.values():
        if "Nombre Establecimiento" in df.columns and "Area" in df.columns:
            pares = df[["Nombre Establecimiento", "Area"]].dropna().drop_duplicates()
            for _, row in pares.iterrows():
                pares_hospital_area.add((row["Nombre Establecimiento"], row["Area"]))

    return sorted(list(pares_hospital_area))

# Extraer todos los hospitales y áreas válidos
hospitales_areas_validos = extraer_hospitales_y_areas_validos(combinados)

# Mostrar los primeros 10 pares para verificar
hospitales_areas_validos[:10]





[('Complejo Hospitalario San José (Santiago, Independencia)',
  '330 - Área Pensionado'),
 ('Complejo Hospitalario San José (Santiago, Independencia)',
  '401 - Área Médica Adulto Cuidados Básicos'),
 ('Complejo Hospitalario San José (Santiago, Independencia)',
  '402 - Área Médica Adulto Cuidados Medios'),
 ('Complejo Hospitalario San José (Santiago, Independencia)',
  '403 - Área Médico-Quirúrgico Cuidados Básicos'),
 ('Complejo Hospitalario San José (Santiago, Independencia)',
  '404 - Área Médico-Quirúrgico Cuidados Medios'),
 ('Complejo Hospitalario San José (Santiago, Independencia)',
  '405 - Área Cuidados Intensivos Adultos'),
 ('Complejo Hospitalario San José (Santiago, Independencia)',
  '406 - Área Cuidados Intermedios Adultos'),
 ('Complejo Hospitalario San José (Santiago, Independencia)',
  '413 - Área Neonatología Cuidados Básicos'),
 ('Complejo Hospitalario San José (Santiago, Independencia)',
  '414 - Área Neonatología Cuidados Intensivos'),
 ('Complejo Hospitalario San

In [63]:
import numpy as np

# Evaluar solo los primeros 30 hospitales y su primera área válida
resultados_todos = {}
pares_a_evaluar = hospitales_areas_validos[:30]

for hospital, area in pares_a_evaluar:
    try:
        print("\n==========================")
        print(f"🏥 Evaluando: {hospital} | Área: {area}")
        print("==========================")
        resultados = pipeline_final_mae_reducido(hospital, area)
        if resultados is not None:
            resultados_todos[f"{hospital} | {area}"] = resultados
    except Exception as e:
        print(f"❌ Error en {hospital} con área {area}: {e}")
        continue

# Crear resumen comparativo de MAE y error porcentual
resumen_comparativo = []

for nombre_completo, resultados in resultados_todos.items():
    if "Error Absoluto" in resultados.columns and "Error Porcentual (%)" in resultados.columns:
        mae = resultados["Error Absoluto"].mean()
        error_pct = resultados["Error Porcentual (%)"].mean()
        resumen_comparativo.append({
            "Hospital | Área": nombre_completo,
            "MAE (camas)": round(mae, 2),
            "Error Porcentual Promedio (%)": round(error_pct, 2)
        })

df_resumen_comparativo = pd.DataFrame(resumen_comparativo).sort_values("MAE (camas)").reset_index(drop=True)

import ace_tools as tools
tools.display_dataframe_to_user(name="Comparación de desempeño (30 hospitales)", dataframe=df_resumen_comparativo)




🏥 Evaluando: Complejo Hospitalario San José (Santiago, Independencia) | Área: 330 - Área Pensionado
🔍 Extrayendo datos para: Complejo Hospitalario San José (Santiago, Independencia) | 330 - Área Pensionado
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
❌ No se pudo preparar el dataset.

🏥 Evaluando: Complejo Hospitalario San José (Santiago, Independencia) | Área: 401 - Área Médica Adulto Cuidados Básicos
🔍 Extrayendo datos para: Complejo Hospitalario San José (Santiago, Independencia) | 401 - Área Médica Adulto Cuidados Básicos
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo con GridSearchCV...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
📌 Mejores parámetros encontrados: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 50}
📊 Evaluando sobre el 20% final de los datos...
✅ MAE (camas): 67.90
✅ RMSE (camas): 96.07
📉 Error promedio porcentual: 3.34%

🏥 Evaluando: Complejo Hospitalario San Jo

ModuleNotFoundError: No module named 'ace_tools'

In [65]:
# Crear tabla resumen con hospital, área, MAE y porcentaje de error

tabla_resumen = []

for nombre_completo, resultados in resultados_todos.items():
    if "Error Absoluto" in resultados.columns and "Error Porcentual (%)" in resultados.columns:
        hospital, area = nombre_completo.split(" | ", 1)
        mae = resultados["Error Absoluto"].mean()
        error_pct = resultados["Error Porcentual (%)"].mean()
        tabla_resumen.append({
            "Hospital": hospital,
            "Área": area,
            "MAE (camas)": round(mae, 2),
            "Error Porcentual Promedio (%)": round(error_pct, 2)
        })

df_tabla_resumen = pd.DataFrame(tabla_resumen).sort_values("MAE (camas)").reset_index(drop=True)

dataframe=df_tabla_resumen

dataframe


,Hospital,Área,MAE (camas),Error Porcentual Promedio (%)
0,"Complejo Hospitalario San José (Santiago, Inde...",Área Cuidados Intensivos Adultos,0.78,0.24
1,Hospital Adalberto Steeger (Talagante),405 - Área Cuidados Intensivos Adultos,0.91,0.51
2,"Complejo Hospitalario San José (Santiago, Inde...",Área Neonatología Cuidados Básicos,5.88,1.38
3,Hospital Adalberto Steeger (Talagante),330 - Área Pensionado,6.05,1.55
4,"Complejo Hospitalario San José (Santiago, Inde...",Área Cuidados Intermedios Adultos,6.56,0.70
5,"Complejo Hospitalario San José (Santiago, Inde...",Área Neonatología Cuidados Intermedios,7.20,0.84
6,"Complejo Hospitalario San José (Santiago, Inde...",413 - Área Neonatología Cuidados Básicos,7.82,1.65
7,"Complejo Hospitalario San José (Santiago, Inde...",Área Obstetricia,8.40,0.23
8,"Complejo Hospitalario San José (Santiago, Inde...",402 - Área Médica Adulto Cuidados Medios,9.36,0.45
9,"Complejo Hospitalario San José (Santiago, Inde...",Área Pensionado,9.68,1.58


In [ ]:
#######################################
#######################################
##DESDE AQUI, NUEVO MODELO ->LightGBM##
#######################################
#######################################

In [66]:
!pip install lightgbm


In [69]:
# Importar LightGBM
from lightgbm import LGBMRegressor

# Nueva versión del pipeline usando LightGBM en vez de XGBoost
def pipeline_lightgbm_mae_reducido(hospital, area):
    print(f"🔍 [LGBM] Extrayendo datos para: {hospital} | {area}")
    df_base = extraer_info_hospital_area_todo_el_año(hospital, area)

    if df_base.empty:
        print("❌ No se encontraron datos para el hospital o área especificada.")
        return None

    print("🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...")
    df_modelo = preparar_dataset_climatico(df_base)

    if df_modelo is None or df_modelo.empty:
        print("❌ No se pudo preparar el dataset.")
        return None

    # Selección de features
    features = [
        'Dias Cama Ocupados',
        'Promedio Cama Disponibles',
        'Numero de Egresos',
        'Mes', 'Trimestre',
        'lag_1', 'lag_2',
        'porcentaje_ocupacion',
        'variacion_disponibles',
        'ocupados_media_movil',
        'promedio_media_movil',
        'egresos_media_movil',
        'tasa_egresos',
        'tasa_ocupacion_promedio',
        'variacion_ocupados',
        'variacion_egresos',
        'ocupados_x_temp',
        'egresos_x_precipitacion',
        'ocupacion_pct_x_tempmin',
        'Temperatura Máxima',
        'Temperatura Mínima',
        'Precipitaciones (suma)'
    ] + [col for col in df_modelo.columns if col.startswith('Viento_')]

    # Separar train/test
    X = df_modelo[features]
    y = df_modelo['Dias Cama Disponibles']
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

    print("🔧 Ajustando modelo LightGBM con GridSearchCV...")
    param_grid = {
        'n_estimators': [50, 100],
        'max_depth': [2, 3],
        'learning_rate': [0.05, 0.1]
    }

    model = LGBMRegressor(n_jobs=1, random_state=42, verbose=-1)

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='neg_mean_absolute_error',
        cv=TimeSeriesSplit(n_splits=3),
        verbose=1
    )
    grid_search.fit(X_train, y_train)
    mejor_modelo = grid_search.best_estimator_

    print("📌 [LGBM] Mejores parámetros encontrados:", grid_search.best_params_)

    # Evaluación
    y_pred = mejor_modelo.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred) ** 0.5
    error_pct = np.mean(np.abs((y_test.values - y_pred) / y_test.values)) * 100

    resultados = pd.DataFrame({
        'Fecha': df_modelo['Fecha'].iloc[-len(y_test):].values,
        'Real': y_test.values,
        'Predicho': y_pred,
        'Error Absoluto': np.abs(y_test.values - y_pred),
        'Error Porcentual (%)': np.abs((y_test.values - y_pred) / y_test.values) * 100
    })

    print(f"✅ [LGBM] MAE (camas): {mae:.2f}")
    print(f"✅ [LGBM] RMSE (camas): {rmse:.2f}")
    print(f"📉 [LGBM] Error porcentual promedio: {error_pct:.2f}%")

    return resultados



In [70]:
# Ejecutamos el pipeline LightGBM con un hospital de ejemplo para comparar con XGBoost
hospital = "Hospital Barros Luco Trudeau (Santiago, San Miguel)"
area = "401 - Área Médica Adulto Cuidados Básicos"

resultados_lgbm = pipeline_lightgbm_mae_reducido(hospital, area)

# Mostrar resultados si están disponibles
if resultados_lgbm is not None:
    resultados_lgbm.head()


🔍 [LGBM] Extrayendo datos para: Hospital Barros Luco Trudeau (Santiago, San Miguel) | 401 - Área Médica Adulto Cuidados Básicos
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo LightGBM con GridSearchCV...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
📌 [LGBM] Mejores parámetros encontrados: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 100}
✅ [LGBM] MAE (camas): 122.76
✅ [LGBM] RMSE (camas): 153.64
📉 [LGBM] Error porcentual promedio: 5.72%


In [71]:
import numpy as np

# Evaluar LightGBM sobre los primeros 30 hospitales y sus áreas
resultados_lgbm_todos = {}
pares_a_evaluar = hospitales_areas_validos[:30]

for hospital, area in pares_a_evaluar:
    try:
        print("\n==========================")
        print(f"🏥 [LGBM] Evaluando: {hospital} | Área: {area}")
        print("==========================")
        resultados = pipeline_lightgbm_mae_reducido(hospital, area)
        if resultados is not None:
            resultados_lgbm_todos[f"{hospital} | {area}"] = resultados
    except Exception as e:
        print(f"❌ Error en {hospital} con área {area}: {e}")
        continue

# Crear tabla resumen de resultados
resumen_lgbm = []

for nombre_completo, resultados in resultados_lgbm_todos.items():
    if "Error Absoluto" in resultados.columns and "Error Porcentual (%)" in resultados.columns:
        hospital, area = nombre_completo.split(" | ", 1)
        mae = resultados["Error Absoluto"].mean()
        error_pct = resultados["Error Porcentual (%)"].mean()
        resumen_lgbm.append({
            "Hospital": hospital,
            "Área": area,
            "MAE (camas)": round(mae, 2),
            "Error Porcentual Promedio (%)": round(error_pct, 2)
        })





🏥 [LGBM] Evaluando: Complejo Hospitalario San José (Santiago, Independencia) | Área: 330 - Área Pensionado
🔍 [LGBM] Extrayendo datos para: Complejo Hospitalario San José (Santiago, Independencia) | 330 - Área Pensionado
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
❌ No se pudo preparar el dataset.

🏥 [LGBM] Evaluando: Complejo Hospitalario San José (Santiago, Independencia) | Área: 401 - Área Médica Adulto Cuidados Básicos
🔍 [LGBM] Extrayendo datos para: Complejo Hospitalario San José (Santiago, Independencia) | 401 - Área Médica Adulto Cuidados Básicos
🧪 Preparando dataset con variables hospitalarias, climáticas y derivadas...
🔧 Ajustando modelo LightGBM con GridSearchCV...
Fitting 3 folds for each of 8 candidates, totalling 24 fits
📌 [LGBM] Mejores parámetros encontrados: {'learning_rate': 0.05, 'max_depth': 2, 'n_estimators': 50}
✅ [LGBM] MAE (camas): 147.25
✅ [LGBM] RMSE (camas): 181.52
📉 [LGBM] Error porcentual promedio: 7.40%

🏥 [LGBM] Evaluando: C

,Hospital,Área,MAE (camas),Error Porcentual Promedio (%)
0,"Complejo Hospitalario San José (Santiago, Inde...",Área Cuidados Intensivos Adultos,5.59,1.66
1,Hospital Adalberto Steeger (Talagante),330 - Área Pensionado,10.44,2.69
2,Hospital Adalberto Steeger (Talagante),405 - Área Cuidados Intensivos Adultos,11.14,6.13
3,"Complejo Hospitalario San José (Santiago, Inde...",Área Pensionado,11.84,1.91
4,"Complejo Hospitalario San José (Santiago, Inde...",413 - Área Neonatología Cuidados Básicos,14.60,2.99
5,"Complejo Hospitalario San José (Santiago, Inde...",Área Cuidados Intermedios Adultos,14.92,1.60
6,"Complejo Hospitalario San José (Santiago, Inde...",414 - Área Neonatología Cuidados Intensivos,16.61,7.28
7,"Complejo Hospitalario San José (Santiago, Inde...",415 - Área Neonatología Cuidados Intermedios,18.20,2.28
8,"Complejo Hospitalario San José (Santiago, Inde...",Área Neonatología Cuidados Intensivos,28.56,7.16
9,"Complejo Hospitalario San José (Santiago, Inde...",Área Neonatología Cuidados Intermedios,28.68,3.32


In [59]:
# Mostrar tabla ordenada
df_resumen_lgbm = pd.DataFrame(resumen_lgbm).sort_values("MAE (camas)").reset_index(drop=True)

dataframe2=df_resumen_lgbm
dataframe2

NameError: name 'resumen_lgbm' is not defined

In [77]:
# Generar tabla resumen de LightGBM (vuelve a correr esta si es necesario)
resumen_lgbm = []

for nombre_completo, resultados in resultados_lgbm_todos.items():
    if "Error Absoluto" in resultados.columns and "Error Porcentual (%)" in resultados.columns:
        hospital, area = nombre_completo.split(" | ", 1)
        mae = resultados["Error Absoluto"].mean()
        error_pct = resultados["Error Porcentual (%)"].mean()
        resumen_lgbm.append({
            "Hospital": hospital,
            "Área": area,
            "MAE (camas)": round(mae, 2),
            "Error Porcentual Promedio (%)": round(error_pct, 2)
        })

df_resumen_lgbm = pd.DataFrame(resumen_lgbm).sort_values("MAE (camas)").reset_index(drop=True)


In [79]:
# Comparar XGBoost vs LightGBM en MAE
df_comparacion = pd.merge(
    df_tabla_resumen,
    df_resumen_lgbm,
    on=["Hospital", "Área"],
    suffixes=("_XGBoost", "_LightGBM")
)

# Determinar modelo ganador
df_comparacion["Modelo Ganador (MAE)"] = df_comparacion.apply(
    lambda row: "XGBoost" if row["MAE (camas)_XGBoost"] < row["MAE (camas)_LightGBM"] else "LightGBM",
    axis=1
)

df_comparacion["Diferencia MAE"] = (
    df_comparacion["MAE (camas)_XGBoost"] - df_comparacion["MAE (camas)_LightGBM"]
).round(2)

dataframe=df_comparacion
dataframe


,Hospital,Área,MAE (camas)_XGBoost,Error Porcentual Promedio (%)_XGBoost,MAE (camas)_LightGBM,Error Porcentual Promedio (%)_LightGBM,Modelo Ganador (MAE),Diferencia MAE
0,"Complejo Hospitalario San José (Santiago, Inde...",Área Cuidados Intensivos Adultos,0.78,0.24,5.59,1.66,XGBoost,-4.81
1,Hospital Adalberto Steeger (Talagante),405 - Área Cuidados Intensivos Adultos,0.91,0.51,11.14,6.13,XGBoost,-10.23
2,"Complejo Hospitalario San José (Santiago, Inde...",Área Neonatología Cuidados Básicos,5.88,1.38,48.89,11.44,XGBoost,-43.01
3,Hospital Adalberto Steeger (Talagante),330 - Área Pensionado,6.05,1.55,10.44,2.69,XGBoost,-4.39
4,"Complejo Hospitalario San José (Santiago, Inde...",Área Cuidados Intermedios Adultos,6.56,0.70,14.92,1.60,XGBoost,-8.36
5,"Complejo Hospitalario San José (Santiago, Inde...",Área Neonatología Cuidados Intermedios,7.20,0.84,28.68,3.32,XGBoost,-21.48
6,"Complejo Hospitalario San José (Santiago, Inde...",413 - Área Neonatología Cuidados Básicos,7.82,1.65,14.60,2.99,XGBoost,-6.78
7,"Complejo Hospitalario San José (Santiago, Inde...",Área Obstetricia,8.40,0.23,59.05,1.60,XGBoost,-50.65
8,"Complejo Hospitalario San José (Santiago, Inde...",402 - Área Médica Adulto Cuidados Medios,9.36,0.45,54.64,2.57,XGBoost,-45.28
9,"Complejo Hospitalario San José (Santiago, Inde...",Área Pensionado,9.68,1.58,11.84,1.91,XGBoost,-2.16


In [ ]:
#######################################
#######################################
##DESDE AQUI, PROBAR NUEVAS FEATURES###
#######################################
#######################################

In [75]:
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd
import numpy as np

# Lista de hospitales a evaluar
hospitales_a_evaluar = [
    "Hospital Barros Luco Trudeau (Santiago, San Miguel)",
    "Hospital Dr. Ernesto Torres Galdames (Iquique)",
    "Complejo Hospitalario San José (Santiago, Independencia)",
    "Hospital Carlos Van Buren (Valparaíso)",
    "Hospital Clínico Herminda Martín (Chillán)",
    "Hospital El Pino (Santiago, San Bernardo)",
    "Hospital de Puerto Montt",
    "Hospital Regional de Antofagasta",
    "Hospital de Coquimbo",
    "Hospital de Talca"
]

# Esta área será usada para todos por simplicidad (puedes personalizar)
area_default = "401 - Área Médica Adulto Cuidados Básicos"

# Función principal ya definida antes, ahora la aplicamos masivamente
def evaluar_modelo_con_estacionalidad(hospital, area):
    df_largo = extraer_info_hospital_area_todo_el_año(hospital, area)
    if df_largo.empty:
        return None

    df_pivot = df_largo.pivot(index='Fecha', columns='Glosa', values='Valor').reset_index()
    columnas_necesarias = ['Dias Cama Disponibles', 'Dias Cama Ocupados',
                           'Promedio Cama Disponibles', 'Numero de Egresos']
    if not all(col in df_pivot.columns for col in columnas_necesarias):
        return None

    df_pivot = df_pivot.sort_values('Fecha')
    df_pivot['Mes'] = df_pivot['Fecha'].dt.month
    df_pivot['Trimestre'] = df_pivot['Fecha'].dt.quarter
    df_pivot['lag_1'] = df_pivot['Dias Cama Disponibles'].shift(1)
    df_pivot['lag_2'] = df_pivot['Dias Cama Disponibles'].shift(2)
    df_pivot['porcentaje_ocupacion'] = df_pivot['Dias Cama Ocupados'] / df_pivot['Dias Cama Disponibles']
    df_pivot['variacion_disponibles'] = df_pivot['Dias Cama Disponibles'].diff()
    df_pivot['ocupados_media_movil'] = df_pivot['Dias Cama Ocupados'].rolling(window=3).mean()
    df_pivot['promedio_media_movil'] = df_pivot['Promedio Cama Disponibles'].rolling(window=3).mean()
    df_pivot['egresos_media_movil'] = df_pivot['Numero de Egresos'].rolling(window=3).mean()
    df_pivot['same_month_last_year'] = df_pivot['Dias Cama Disponibles'].shift(12)
    df_pivot['hist_avg_mes'] = df_pivot.groupby('Mes')['Dias Cama Disponibles'].transform('mean')
    df_modelo = df_pivot.dropna().copy()

    features = [
        'Dias Cama Ocupados',
        'Promedio Cama Disponibles',
        'Numero de Egresos',
        'Mes', 'Trimestre',
        'lag_1', 'lag_2',
        'porcentaje_ocupacion',
        'variacion_disponibles',
        'ocupados_media_movil',
        'promedio_media_movil',
        'egresos_media_movil',
        'same_month_last_year',
        'hist_avg_mes'
    ]

    X = df_modelo[features]
    y = df_modelo['Dias Cama Disponibles']
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

    tscv = TimeSeriesSplit(n_splits=3)
    param_grid = {
        'n_estimators': [50, 100],
        'max_depth': [2, 3],
        'learning_rate': [0.05, 0.1]
    }
    modelo_xgb = XGBRegressor(n_jobs=1, random_state=42)
    grid_search = GridSearchCV(modelo_xgb, param_grid, scoring='neg_mean_absolute_error', cv=tscv, verbose=0)
    grid_search.fit(X_train, y_train)

    mejor_modelo = grid_search.best_estimator_
    y_pred = mejor_modelo.predict(X_test)

    resultados = pd.DataFrame({
        'Fecha': df_modelo['Fecha'].iloc[-len(y_test):].values,
        'Real': y_test.values,
        'Predicho': y_pred,
        'Error Absoluto': np.abs(y_test.values - y_pred),
        'Error Porcentual (%)': np.abs((y_test.values - y_pred) / y_test.values) * 100
    })

    resumen = {
        "Hospital": hospital,
        "Área": area,
        "MAE": mean_absolute_error(y_test, y_pred),
        "Error %": np.mean(resultados['Error Porcentual (%)'])
    }

    return resumen

# Evaluar para todos los hospitales
resultados_multiples = []
for hosp in hospitales_a_evaluar:
    resultado = evaluar_modelo_con_estacionalidad(hosp, area_default)
    if resultado:
        resultados_multiples.append(resultado)

dataframe_result=df_resultados_hospitales

dataframe_result







⚠️ No se encontraron datos para ese hospital y área.
⚠️ No se encontraron datos para ese hospital y área.
⚠️ No se encontraron datos para ese hospital y área.
⚠️ No se encontraron datos para ese hospital y área.


,Hospital,Área,MAE,Error %
0,"Hospital Barros Luco Trudeau (Santiago, San Mi...",401 - Área Médica Adulto Cuidados Básicos,60.805017,2.845284
1,"Complejo Hospitalario San José (Santiago, Inde...",401 - Área Médica Adulto Cuidados Básicos,194.859090,9.504034
2,Hospital Carlos Van Buren (Valparaíso),401 - Área Médica Adulto Cuidados Básicos,38.057153,1.452190
3,Hospital Clínico Herminda Martín (Chillán),401 - Área Médica Adulto Cuidados Básicos,205.300684,10.085671
4,"Hospital El Pino (Santiago, San Bernardo)",401 - Área Médica Adulto Cuidados Básicos,19.791855,1.921970
5,Hospital de Puerto Montt,401 - Área Médica Adulto Cuidados Básicos,197.631835,65.040586


In [77]:
# Crear tabla resumen desde resultados del nuevo modelo con estacionalidad
tabla_resumen_estacionalidad = []

for nombre_completo, resultados in resumen.items():
    if isinstance(resultados, pd.DataFrame) and \
       "Error Absoluto" in resultados.columns and \
       "Error Porcentual (%)" in resultados.columns:
        
        hospital, area = nombre_completo.split(" | ", 1)
        mae = resultados["Error Absoluto"].mean()
        error_pct = resultados["Error Porcentual (%)"].mean()
        
        tabla_resumen_estacionalidad.append({
            "Hospital": hospital,
            "Área": area,
            "MAE (camas)": round(mae, 2),
            "Error Porcentual Promedio (%)": round(error_pct, 2)
        })

df_tabla_resumen_estacionalidad = (
    pd.DataFrame(tabla_resumen_estacionalidad)
    .sort_values("MAE (camas)")
    .reset_index(drop=True)
)

dataframe=df_tabla_resumen_estacionalidad


df_tabla_resumen_estacionalidad


NameError: name 'resumen' is not defined